##  SURNAME: Benvenuto NAME: Giulia
#### **I cleared all the outputs because otherwise the size of the notebook was too big to be uploaded on aulaweb.**

In [ ]:
import numpy as np
import cv2 as cv2
from scipy import signal
import matplotlib.pyplot as plt
from visualize_flow import flow_to_color
import time
%matplotlib inline

##  Lucas-Kanade optical flow algorithm

In this lab we deepen our understanding on the Lucas Kanade (LK) optical flow algorithm. 

<ol>
    <li>Part 1: we go into the <b>implementation details</b> of a single scale estimation (the one we saw in class) </li>
    <li>Part 2 (optional): we will use the <b>pre-implemented </b> algorithm (OpenCV) for a multi-resolution pyramidal version.</li>
</ol>
 
### Part1 - Single Scale LK 

In [ ]:
# we start by loading a pair of adjacent frames
#img1= cv2.imread('Data/stennis/stennis_002.ppm', cv2.IMREAD_GRAYSCALE)
#img2= cv2.imread('Data/stennis/stennis_003.ppm', cv2.IMREAD_GRAYSCALE)
img1= cv2.imread('Data/sphere/sphere.12.ppm', cv2.IMREAD_GRAYSCALE)
img2= cv2.imread('Data/sphere/sphere.13.ppm', cv2.IMREAD_GRAYSCALE)

plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.show()

<b> Fill in the missing details on the Lucas_Kanade function, after you analyse and understand what's already there: </b>

In [ ]:
# this function computes the optical flow between 2 frames

# Takes as inputs two grayscale images im1 and im2 taken at times t1 and t2, a window size window_size (odd) that controls the size 
# of the pixel neighbourhoodand tau that is a threshould on the "quality" of the neighbourhood
def Lucas_Kanade(im1, im2, window_size, tau):
    
    # Spatio-temporal derivative kernels: the derivative of an image is a measure of how quickly the image intensity changes in 
    # different directions. Therefore, the derivative kernel highlights the areas where the image intensity changes rapidly in a 
    # particular direction, here we use these kernels to find the corners.
    kernel_x = np.array([[-1., 1.], [-1., 1.]])*.25 
    kernel_y = np.array([[-1., -1.], [1., 1.]])*.25
    kernel_t = np.array([[1., 1.], [1., 1.]])*.25
    
    # window_size is odd, all the pixels with offset in between [-w, w] are inside the window
    w = int(window_size/2) 
    
    # Normalization of the image pixel values to a range of 0 - 1.
    I1g = im1 / 255. # normalize pixels
    I2g = im2 / 255. # normalize pixels
    
    
    # Implement Lucas Kanade
    # for each point, calculate I_x, I_y, I_t
    
    # Applying the kernels convolving them with the image and we get back the derivatives in space (fx and fy) and time (ft)
    mode = 'same'
    fx = signal.convolve2d(I1g, kernel_x, boundary='symm', mode=mode)# + signal.convolve2d(I2g, kernel_x, boundary='symm', mode=mode)
    fy = signal.convolve2d(I1g, kernel_y, boundary='symm', mode=mode)# + signal.convolve2d(I2g, kernel_y, boundary='symm', mode=mode) 
    ft = signal.convolve2d(I1g, kernel_t, boundary='symm', mode=mode) + signal.convolve2d(I2g, -kernel_t, boundary='symm', mode=mode)
    
    # Initializing the matrices u and v of the same shape of the image in which the algorithm will put the vertical and horizontal components of the optical flow
    u = np.zeros(I1g.shape)
    v = np.zeros(I1g.shape)
    # within window window_size * window_size
    
    
    for i in range(w, I1g.shape[0] - w):
        for j in range(w, I1g.shape[1] - w):
            
            # For every pixel in the image we take the derivatives in its neighbourhood 
            # Initializes the variables Ix, Iy, and It by flattening the spatio-temporal derivatives over the window "w"
            Ix = fx[i-w:i+w+1, j-w:j+w+1].flatten()
            Iy = fy[i-w:i+w+1, j-w:j+w+1].flatten()
            It = np.array(ft[i-w:i+w+1, j-w:j+w+1].flatten())
            
            # Build the matrix A and the vector b to resolve the linear system Au=b
            A = np.transpose(np.array([Ix, Iy]))
            b = -It
            
            # compute the smallest eigenvalue of the autocorrelation matrix.
            # The eigenvalues of A^T A are related to the gradient of the image and are used to determine if the optical flow calculation is reliable. 
            # If the minimum eigenvalue is greater than or equal to a threshold "tau", then the calculation is considered reliable, 
            # and the optical flow is calculated. Otherwise, the optical flow is set to zero.
            if np.min(abs(np.linalg.eigvals(np.matmul(A.T, A)))) >= tau: 
                
                # Solve the linear system with the pseudo-inverse.
                # It could happen that the resulting equations are overdetermined, meaning that there are more equations than unknowns. For such a system the 
                # matrix inverse method will not work because the A matrix is not square.
                # In order to solve the overdetermined system of equations, the Lucas-Kanade algorithm uses the pseudo inverse of the coefficient matrix.
                nu = np.matmul(np.linalg.pinv(A), b) 
                u[i,j] = nu[0] # adding the horizontal component of the optical flow
                v[i,j] = nu[1] # adding the vertical component of the optical flow
      
    
    # Return thhe velocity vector of the tracked feature.
    # Returns the horizontal and vertical components of the optical flow u and v
    return (u,v) 

### Lucas Kanade Algorithm 
This algorithm is a widely used in computer vision for estimating the optical flow of an image. Optical flow is the apparent motion of objects in an image so it is the answer to the problem of estimating the apparent motion of the brightness pattern of the image.

The Lucas-Kanade algorithm assumes that the optical flow is locally constant, so it assumes that the motion between two nearby points in an image is similar. Following the notation of the slides we can say that u is constant in a small neighbourhood of a point.

The algorithm works by taking a small window around a pixel in the first image, and searching for a corresponding window in the second image that is most similar to the first one. The displacement between the two is assumed to be the optical flow. The optical flow minimizes the sum-squared error of the brightness constancy equations for each pixel in a window.



<br>
Call the previously defined method

In [ ]:
start = time.time()
[u,v] = Lucas_Kanade(img1, img2, 7, 0.001)
print("Elapsed time is %d"%(time.time() - start))

Let's visualize the results of the flow field. We may try the quiver function first

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
print(u.shape)
print(v.shape)
xaxis = list(np.arange(img1.shape[0]))
yaxis = list(np.arange(img1.shape[1]))
#plt.xlim(0, 352)
#plt.ylim(240,0)
plt.quiver(u,v)
plt.show()

In [ ]:
# plt.quiver??

We may also visualize the flow field using a color-coding algorithm ( See https://github.com/tomrunia/OpticalFlow_Visualization)

In [ ]:
flow = np.stack([u,v], axis=2)
flow_color = flow_to_color(flow, convert_to_bgr=False)
plt.imshow(flow_color)
plt.show()

###   Analysis:

- Go back to the function Lucas_Kanade and a comment to explain each line of code  
- In particular, can you explain this? <tt> np.min(abs(np.linalg.eigvals(np.matmul(A.T, A)))) >= tau: </tt>
**I commented out all the function line by line in the code cell**

### Experiments:
- Try different window size and thresholds tau 
- Try different frames (also introducing some temporal gap between them)  
- Try both sphere and tennis datasets
- What do you observe ? Do you see any specific limits in the very simple LK method?


### Different thresholds tau

In [ ]:
print("Tau values: " + str(np.logspace(-6, -1, num = 6)))

In [ ]:
img1 = cv2.imread('Data/sphere/sphere.12.ppm',cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/sphere/sphere.13.ppm',cv2.IMREAD_GRAYSCALE)

fig, axs = plt.subplots(2, 3, figsize=(18, 8))

for ax, tau in zip(axs.flatten(), np.logspace(-6, -1, num = 6)):
    [u,v] = Lucas_Kanade(img1, img2, 7, tau)
    ax.set_title("Tau: {}".format(tau))
    ax.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    ax.quiver(u, v)

Visualize the flow field using a color-coding algorithm

In [ ]:
img1 = cv2.imread('Data/sphere/sphere.12.ppm',cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/sphere/sphere.13.ppm',cv2.IMREAD_GRAYSCALE)

fig, axs = plt.subplots(2, 3, figsize=(18, 8))

for ax, tau in zip(axs.flatten(), np.logspace(-6, -1, num = 6)):
    
    [u,v] = Lucas_Kanade(img1, img2, 7, tau)
    ax.set_title("Tau: {}".format(tau))
    flow = np.stack([u,v],axis=2)
    flow_color = flow_to_color(flow, convert_to_bgr=False)
    ax.imshow(flow_color)

**Observations about tau:** 

I tried to visualize the flow field for different values of tau. 
- If **tau is small** we have a bigger number of points in which the optical flow is computed, this result in a more detailed flow field that is better able to capture fine-scale variations in the image. However, this can also make the flow filed more susceptible to noise.
- If **tau is big** we have a smaller number of points in which the optical flow is computed, this result in a smoother and less detailed flow field. I think that having a big value for tau can be useful when dealing with noisy images because we can prevent "overfitting" to the noise. 

### Different window size

In [ ]:
print("Window values: " + str(np.round(np.linspace(1, 25, num=8))))

In [ ]:
img1 = cv2.imread('Data/sphere/sphere.12.ppm',cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/sphere/sphere.13.ppm',cv2.IMREAD_GRAYSCALE)

fig, axs = plt.subplots(2, 4, figsize=(18, 8))

for ax, w in zip(axs.flatten(), np.round(np.linspace(1, 25, num=8))):
    
    [u,v] = Lucas_Kanade(img1, img2, w, 1e-5)
    ax.set_title("Window size: {}".format(w))
    flow = np.stack([u,v],axis=2)
    flow_color = flow_to_color(flow, convert_to_bgr=False)
    ax.imshow(flow_color)

**Observations about the window size:**

The window size determines the size of the neighborhood around each pixel that is used to estimate the optical flow. The given window size was 7, I tried both to increase it to 11 and to decrease it to 5.
- If the **window size is bigger** we will get as result a more global estimate of the optical flow because the algorithm will consider a larger neighborhood around each pixel. I think that increasing the window size can be useful if we are dealing with large-scale motion but if we're dealing with fine-scale motion it may make the algorithm less sensitive.
- If the **window size is smaller** we will get as result a more global estimate of the optical flow because the algorithm will consider a smaller neighborhood around each pixel. I think that decresing the window size can be useful if we are dealing with fine-scale motion, but it may also make the algorithm more susceptible to noise.

### Different frames (introducing temporal gap between them)

In [ ]:
img1 = cv2.imread('Data/sphere/sphere.1.ppm', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/sphere/sphere.15.ppm', cv2.IMREAD_GRAYSCALE)

start = time.time()
[u,v] = Lucas_Kanade(img1, img2, 7, 0.001)
print("Elapsed time is %d"%(time.time() - start))

fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(8, 6))
ax[0].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[0].quiver(u,v)

flow = np.stack([u,v], axis=2)
flow_color = flow_to_color(flow, convert_to_bgr=False)
ax[1].imshow(flow_color)

**Observations about different frames with temporal gap:**
If we compare two frames with some temporal gap between them (frames that are not consecutive in time), the resulting optical flow field represents the average motion of the objects in the scene over the time interval between the frames.

### Tennis Dataset

In [ ]:
img1 = cv2.imread('Data/stennis/stennis_001.ppm', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/stennis/stennis_020.ppm', cv2.IMREAD_GRAYSCALE)

plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
start = time.time()
[u,v] = Lucas_Kanade(img1, img2, 7, 0.001)
print("Elapsed time is %d"%(time.time() - start))

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
print(u.shape)
print(v.shape)
xaxis = list(np.arange(img1.shape[0]))
yaxis = list(np.arange(img1.shape[1]))
#plt.xlim(0, 352)
#plt.ylim(240,0)
plt.quiver(u,v)
plt.show()

In [ ]:
flow = np.stack([u,v], axis=2)
flow_color = flow_to_color(flow, convert_to_bgr=False)
plt.imshow(flow_color)
plt.show()

**Observations about the Tennis Dataset:**

The algorithm manage to detect the movement of the tennis ball. But it seems strange to me that in the tennis ball there are so many arrows pointing downwards since the ball is moving upwards.

### Optional Part2 - Pyramidal LK
<i>We did not cover the Pyramidal extension during the OF class; this part should be considered as additional and optional material. </i>
<br>
Now let's take a look at the implementation provided in OpenCV of a pyramid Lucas-Kanade Sparse optical flow

The algorithm evaluates optical flow on sparse points (corners) in order to avoid the ill-posed inversion of A'A.
Additionally, Optical flow is calculated and combined on different scales to handle large-displacement

In [ ]:
img1= cv2.imread('Data/sphere/sphere.12.ppm', cv2.IMREAD_GRAYSCALE)
img2= cv2.imread('Data/sphere/sphere.13.ppm', cv2.IMREAD_GRAYSCALE)

In [ ]:
#parameters of the corner detection procedure
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.1,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

1- Call the function that detects the key-points (Shi-Tomasi corners) from the first frame 
2- Call LK Flow algorithm which returns the positions of these key points in the second frame

In [ ]:
p0 = cv2.goodFeaturesToTrack(img1, mask = None, **feature_params)
p1, st, err = cv2.calcOpticalFlowPyrLK(img1, img2, p0, None, **lk_params)

Draw vectors to connect points from the first frame and the second frame to visualize the motion vectors

In [ ]:
good_new = p1[st==1]
good_old = p0[st==1]
mask = np.zeros_like(img1)
print(good_old.shape)
for i,(new,old) in enumerate(zip(p1,p0)):
    a,b = np.int32(new.ravel())
    c,d = np.int32(old.ravel())
    mask = cv2.line(mask, (a,b),(c,d), [255,255,0], 2)
img2 = cv2.add(img2,mask)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

### Tennis Dataset

In [ ]:
img1 = cv2.imread('Data/stennis/stennis_001.ppm', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/stennis/stennis_020.ppm', cv2.IMREAD_GRAYSCALE)

In [ ]:
#parameters of the corner detection procedure
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.1,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


In [ ]:
p0 = cv2.goodFeaturesToTrack(img1, mask = None, **feature_params)
p1, st, err = cv2.calcOpticalFlowPyrLK(img1, img2, p0, None, **lk_params)

In [ ]:
good_new = p1[st==1]
good_old = p0[st==1]
mask = np.zeros_like(img1)
print(good_old.shape)
for i,(new,old) in enumerate(zip(p1,p0)):
    a,b = np.int32(new.ravel())
    c,d = np.int32(old.ravel())
    mask = cv2.line(mask, (a,b),(c,d), [255,255,0], 2)
img2 = cv2.add(img2,mask)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))